In [ ]:
# For the first time running, install following pakages (Remove # below)
 # Pkg.add("VoronoiFVM")
 # Pkg.add("ExtendableGrids")
 # Pkg.add("PyPlot")
 # Pkg.add("Plots")
 # Pkg.add("DataFrames")
 # Pkg.add("CSV")

In [ ]:
# Guide (based on Julia v1.5.1 and VoronoiFVM.jl v0.10.2)
 # Input parameter/Set shelf conditions
 # Run
 # Read Bottom_T_HFD.csv => Find time where sublimation temperature reached (t1)
 # Set inival[3,i] as new shelf temperature (inival[3,i] = inival[1,i] + t1 * Tshelf_r(Kelvin/10sec) * 60 sec/min * 60 min/hr)
 # Re-run
 # Read Position_HFD.csv => Find time where length reached (t2)
 # Drying time = t1 + t2

In [ ]:
using VoronoiFVM
using ExtendableGrids
using PyPlot
using Plots

Length=0.042;
edge_n=300; # number of edges
X1=collect(0.0:Length/edge_n:Length)
grid=ExtendableGrids.simplexgrid(X1)
ExtendableGrids.plot(grid,Plotter=PyPlot)

In [ ]:
# Parameter
density1=63;
density2=917;
cp2=1967.8;
pbwi=0.04;
picei=1-pbwi;
pw=0.92;
k2=2.30;
Tsub=256.15; # Sublimation temperature
Hsub=2.840*10^6;
htc=65; # Heat transfer coefficient # W/m^2K # Hottot 2007
Hw=242345;
Tshelf_i=236.85; # Initial shelf temperature
Tshelf_f=281.85; # Final shelf temperature
Tshelf_r=1/6;    # Shelf temperature rising per 10s

In [ ]:
function storage!(f,u,node) # Storage term  # 1:T2, 2:X(t), 3:Tshelf
        f[1]=density2*cp2*u[1]
        f[2]=(density2-density1)*(Hsub*picei)*u[2]
        f[3]=u[3]
end

In [ ]:
function flux!(f,u,edge) # Flux term
        f[1]=-k2*(u[1,2]-u[1,1])
        f[2]=0
        f[3]=0
end

In [ ]:
function reaction!(f,u,node) # Reaction term
        f[1]=-Hw*pbwi
        f[2]=-Hw*Length*pw-htc*(u[3]-Tsub)
        f[3]=-1/60
end

In [ ]:
physics=VoronoiFVM.Physics(num_species=3,flux=flux!,storage=storage!,reaction=reaction!)

In [ ]:
system=VoronoiFVM.DenseSystem(grid,physics)
enable_species!(system,1,[1]) # System, ispec::Integer, regions::AbstractArray{T,1}
enable_species!(system,2,[1])
enable_species!(system,3,[1])

In [ ]:
inival=unknowns(system)
solution=unknowns(system)
for i=1:num_nodes(grid)
    inival[1,i]=236.85      # Initial temperature
    inival[2,i]=0           # Initial position
    inival[3,i]=236.85+33.2 # Shelf temp. at the initial sublimation point
end

In [ ]:
using Plots
using DataFrames
using CSV
df_a=DataFrame()
df_b=DataFrame()
df_c=DataFrame()

let
t=0.0; del_t=10.0
    @gif for i=1:360*3
         t=t+del_t
            
        if i<(Tshelf_f-Tshelf_i)/Tshelf_r+1
        boundary_neumann!(system,1,1,htc*(Tshelf_i+Tshelf_r*i-solution[1,1])/k2)       
        end
        
        if i>(Tshelf_f-Tshelf_i)/Tshelf_r
        boundary_neumann!(system,1,1,htc*(Tshelf_f-solution[1,1])/k2)
        end
        
        if solution[3,1]>Tshelf_f-0.001
        boundary_dirichlet!(system,3,1,Tshelf_f)
        end
        
        
        push!(df_a,(A=t/3600,B=solution[1,300]-273.15)) # Interface temperature
        CSV.write("Interface_T_HFD.csv",df_a,writeheader=false)
        
        push!(df_b,(A=t/3600,B=solution[1,1]-273.15)) # Bottom temperature
        CSV.write("Bottom_T_HFD.csv",df_b,writeheader=false)
        
        push!(df_c,(A=t/3600,B=solution[2,1])) # Interface position
        CSV.write("Position_HFD.csv",df_c,writeheader=false)
        
        solve!(solution,inival,system,tstep=del_t)
    
        @views begin
        P1=heatmap(reshape(solution[1,:],length(X1),1),colorbar=:right,color=:viridis,clim=(220,260),yflip=false)
        P2=Plots.plot(solution[1,:],ylim=(220,260),yflip=false)
        P3=Plots.plot(solution[2,:],ylim=(0,0.042),yflip=false)
        P4=Plots.plot(solution[3,:],ylim=(236,286),yflip=false)
        P=Plots.plot(P1,P2,P3,P4,layout=(1,4))
        end
        inival.=solution; del_t*=1.00
   end
end